In [61]:
import requests
import urllib
import re
from bs4 import BeautifulSoup
from googlesearch import search
import pandas as pd
import numpy as np
from fake_useragent import UserAgent
import time
import pickle

In [ ]:
cities_data = pd.read_csv("C:\\Users\\Crist\\AppData\\Local\\Temp\\American-Community-Survey-Project\\cal_cities_lat_long.csv")

In [ ]:
city_list = cities_data.Name.tolist()
list_of_cities = []
for city in city_list:
    new = city.replace(' ','-')
    list_of_cities.append(new)

In [ ]:
list_of_cities

In [ ]:
clean_links = []
missing_cities = []

In [ ]:
for city in list_of_cities:
    #only get links for cities not in clean_links
        backup_url = "https://www.towncharts.com/California/Housing/{}-city-CA-Housing-data.html".format(city)
        query  = "'https://www.towncharts.com/California/Housing/{}-city-CA-Housing-data.html'".format(city)
        query = urllib.parse.quote_plus(query) # Format into URL encoding
        number_result = 3
        
        ua = UserAgent()
        
        google_url = "https://www.google.com/search?q=" + query + "&num=" + str(number_result)
        t0 = time.time()
        response = requests.get(google_url, {"User-Agent": ua.random})
        response_delay = time.time() - t0
        time.sleep(10*response_delay)
        soup = BeautifulSoup(response.text, "html.parser")
        
        result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})
        
        links = []
        for r in result_div:
            
            try:
                link = r.find('a', href = True)

                if link != '':
                    links.append(link['href'])
            except:
                continue
        flag = True
        for i, l in enumerate(links):
            clean = re.search('\/url\?q\=(.*)\&sa',l)
            if clean is None:
                continue
            elif city in clean.group(1):
                clean_links.append(clean.group(1))
                flag = False
        if flag:
            missing_cities.append(city)
            clean_links.append(backup_url)

Not perfect so we will:
    *remove identical strings
    *remove strings that have the word county in them
    *track http error 404 if our self made url fails

Pickle the output

In [ ]:
# outfile = open('my_clean_links','wb')
# pickle.dump(clean_links,outfile)
# outfile.close()

# outfile = open('unfound_urls','wb')
# pickle.dump(missing_cities,outfile)
# outfile.close()

In [34]:

# infile = open('my_clean_links','rb')
# clean_links = pickle.load(infile)
# infile.close()

# infile = open('unfound_urls','rb')
# unfound_urls = pickle.load(infile)
# infile.close()

In [40]:
#remove county results
without_counties = pd.Series(clean_links)
without_counties = without_counties.mask(pd.Series(clean_links).str.contains('County'))
without_counties = without_counties.dropna()

In [41]:
#delete duplicates
city_urls = without_counties.drop_duplicates()

A quick overview shows economy was gathered for avalon
and some cities have cdd rather than the city

In [42]:
economy_indexes = city_urls[city_urls.str.contains('Economy')].index

In [43]:
CCD_indexes = city_urls[city_urls.str.contains('CCD')].index

In [48]:
for i in economy_indexes:
    print(city_urls[i])
for i in CCD_indexes:
    print(city_urls[i])

http://www.towncharts.com/California/Economy/Avalon-city-CA-Economy-data.html
http://www.towncharts.com/California/Economy/Blythe-city-CA-Economy-data.html
https://www.towncharts.com/California/Economy/Chowchilla-city-CA-Economy-data.html
http://www.towncharts.com/California/Housing/Barstow-CCD-CA-Housing-data.html
http://www.towncharts.com/California/Housing/Bishop-CCD-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Carpinteria-CCD-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Chico-CCD-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Coachella-Valley-CCD-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Central-Colusa-CCD-CA-Housing-data.html


In [49]:
indexes_to_replace = pd.Series.tolist(economy_indexes)+pd.Series.tolist(CCD_indexes)
indexes_to_replace

[22, 44, 79, 29, 41, 68, 74, 87, 92]

In [50]:
towns_to_fix = pd.Series(['Avalon','Blythe','Chowchilla','Barstow','Bishop','Carpinteria','Chico','Coachella','Colusa'],index = indexes_to_replace)
towns_to_fix

22         Avalon
44         Blythe
79     Chowchilla
29        Barstow
41         Bishop
68    Carpinteria
74          Chico
87      Coachella
92         Colusa
dtype: object

In [51]:
cop_city_urls = city_urls.copy()

In [52]:
for i,v in towns_to_fix.items():
    cop_city_urls[i] = "https://www.towncharts.com/California/Housing/{}-city-CA-Housing-data.html".format(v)
    print(cop_city_urls[i])

https://www.towncharts.com/California/Housing/Avalon-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Blythe-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Chowchilla-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Barstow-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Bishop-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Carpinteria-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Chico-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Coachella-city-CA-Housing-data.html
https://www.towncharts.com/California/Housing/Colusa-city-CA-Housing-data.html


In [54]:
len(cop_city_urls)

466

In [55]:
#delete duplicates if accidentally created any
searchurls = cop_city_urls.drop_duplicates()
len(searchurls)

461

A quick look through of the first 100 shows that Burbank has a duplicate with CDP and Bishop city has a duplicate or https vs http

In [60]:
display(searchurls[searchurls.str.contains('CDP')])
searchurls[searchurls.str.contains('Bishop')]

52    https://www.towncharts.com/California/Housing/...
dtype: object

40    http://www.towncharts.com/California/Housing/B...
41    https://www.towncharts.com/California/Housing/...
dtype: object

In [62]:
searchurls = searchurls.drop(labels = [40,52])
searchurls = searchurls.reset_index(drop=True)
len(searchurls)

459

Now we actually scrape the desired attributes

In [ ]:
def searchpattern(pattern,resultset):
    '''
    pattern:regex pattern
    resultset: A beautiful soup result set
    output housing variable of interest type string
    '''
    result = [i for i in resultset if re.search(pattern,i)]
    if result == []:
        result  = None
    else:
        result = re.findall(pattern,result[0])
    return result

city_name = re.search('(.*)\sCalifornia',housing_text_section1[0]).group(1)

#for variables need to remove everything but numbers
#figure1 
housing_units = searchpattern('it\shas\s(\d.*?\d)\shousing\sunits\s',housing_text_section1)
#figure2 
housing_density = searchpattern('it\shas\s(\d.*?\d)\shousing\sdensity\s',housing_text_section1)
#figure4 
change_hunits = searchpattern('it\shas\s(.*?%)\schange\sin\s',housing_text_section1)
#figure5
percent_of_rent_to_total = searchpattern('it\shas\s(\d.*?%)\sRenter\sPercent\s',housing_text_section1)
#figure 6
owned_homes = searchpattern('it\shas\s(\d.*?%)\sOwner\sPercent\s',housing_text_section1)
#figure 7 median home value
med_homeval = searchpattern('it\shas\s(.*?\d)\smedian\shome\svalue\s',housing_text_section2)
#figure 8 median rental rate
med_rental_rate = searchpattern('it\shas\s(.*?\d)\srental\srates\s',housing_text_section2)
#figure 10 median owner costs
med_owner_cost = searchpattern("it\shas\s(.*?\d)\smedian\sownership\scost\s",housing_text_section2)
#figure 11
med_OwnCostas_Perc_of_Income = searchpattern('median\sowner\scost\sas\sa\spercent\sof\stotal\shousehold\sincome\sand\sit\shas\s(\d.*?%)',housing_text_section2)

#figure 12 median home value as a percent of a median workers earnings
med_HCasPerc_of_medearn = searchpattern("it\shas\s(\d.*?%)\smedian\shome\svalue\sas\s",housing_text_section2)
#figure14
med_hcost_ownmortg=searchpattern('depicts.*\$(.*?\d)\smedian',housing_text_section2)
#figure 15
med_hcost_own_wo_mortg = searchpattern('are.*\$(.*?\d)\smedian\shousing\s',housing_text_section2)
#figure 17
hcost_as_perc_of_hincome_ownmortg= searchpattern('with\smortgages.*\s(\d.*?%)\scost',housing_text_section2)
#figure18
hcost_as_perc_of_hincome_womortg= searchpattern('This\sanalysis.*\s(\d.*?%)\scost',housing_text_section2)
#figure 21 **
med_real_estate_taxes =searchpattern('it\shas\s(.*?\d)\smedian\sreal\sestate\s',housing_text_section2)
#figure 24 biggie see below

#figure 28 format 3.0
family_members_per_hunit = searchpattern('it\shas\s(.*?\d)\sfamily\smembers\sper\s',housing_text_section3)
#figure 29 4.5
median_numof_rooms = searchpattern('it\shas\s(.*?\d)\saverage\snumber\sof\s',housing_text_section3)
#figure 30
median_year_house_built = searchpattern('it\shas\s(.*?\d)\syear\sbuilt\swhich\s',housing_text_section3)
#figure 36
household_size_of_howners = searchpattern('it\shas\s(.*?\d)\shomeowner\shousehold\ssize\s',housing_text_section3)
#figure 37
household_size_for_renters = searchpattern('it\shas\s(.*?\d)\srenter\shousehold\ssize\s',housing_text_section3)
#figure 40
med_year_moved_in_for_owners = searchpattern('it\shas\s(.*?\d)\smedian\syear\sthat\sa\shome',housing_text_section3)
#figure41
med_year_renter_moved_in = searchpattern('it\shas\s(.*?\d)\smedian\syear\sthat\sa\srenter',housing_text_section3)

In [349]:
#without_counties.str.extract(r'Housing\/(.*)\-CA',expand=True)

,0
0,Agoura-Hills-city
1,Alameda-city
3,Albany-city
4,Alhambra-city
5,Aliso-Viejo-city
6,American-Canyon-city
7,Anaheim-city
